# Label Studio to GLINER2 Training Converter

This notebook converts Label Studio NER annotations to GLINER2 JSONL training format.

**Input**: Label Studio JSON export with NER annotations  
**Output**: GLINER2-compatible JSONL file for training

**File Paths:**
- Input: `test_validated_NER_annotations.json` (Label Studio export)
- Output: `gliner2_training_data.jsonl` (GLINER2 training format)

In [2]:
import json
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Set

# File paths
INPUT_FILE = r"C:\Users\vducatte\OneDrive - UGent\Documents\BELHISFIRM\NER\gliner2-project\test_validated_NER_annotations.json"
OUTPUT_FILE = r"C:\Users\vducatte\OneDrive - UGent\Documents\BELHISFIRM\NER\gliner2-project\gliner2_training_data.jsonl"

print("✓ Configuration loaded")
print(f"  Input:  {INPUT_FILE}")
print(f"  Output: {OUTPUT_FILE}")

✓ Configuration loaded
  Input:  C:\Users\vducatte\OneDrive - UGent\Documents\BELHISFIRM\NER\gliner2-project\test_validated_NER_annotations.json
  Output: C:\Users\vducatte\OneDrive - UGent\Documents\BELHISFIRM\NER\gliner2-project\gliner2_training_data.jsonl


In [3]:
def load_labelstudio_annotations(filepath: str) -> List[dict]:
    """
    Load Label Studio JSON export file.
    
    Args:
        filepath: Path to Label Studio JSON export
        
    Returns:
        List of task dictionaries with annotations
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if not isinstance(data, list):
        raise ValueError(f"Expected a list of tasks, got {type(data)}")
    
    print(f"✓ Loaded {len(data)} tasks from Label Studio export")
    return data

# Load the data
tasks = load_labelstudio_annotations(INPUT_FILE)

✓ Loaded 2 tasks from Label Studio export


In [4]:
def extract_entities_from_labelstudio(task: dict) -> tuple[str, Dict[str, List[str]], Set[str]]:
    """
    Extract entities from a Label Studio task.
    
    Args:
        task: Label Studio task dictionary
        
    Returns:
        Tuple of (text, entities_dict, entity_types_set)
    """
    # Get the text
    text = task.get('data', {}).get('text', '')
    
    if not text:
        return None, {}, set()
    
    # Get annotations (prefer human annotations over predictions)
    annotations_list = task.get('annotations', [])
    
    if not annotations_list:
        # Fallback to predictions if no human annotations
        predictions = task.get('predictions', [])
        if predictions:
            annotations_list = [predictions[0]]
        else:
            return text, {}, set()
    
    # Use the first annotation (or most recent)
    annotation = annotations_list[0]
    results = annotation.get('result', [])
    
    # Group entities by label
    entities_by_label = defaultdict(list)
    entity_types = set()
    
    for item in results:
        # Only process label annotations (not relations)
        if item.get('type') != 'labels':
            continue
        
        value = item.get('value', {})
        entity_text = value.get('text', '')
        labels = value.get('labels', [])
        
        if not entity_text or not labels:
            continue
        
        # Use the first label
        label = labels[0]
        entity_types.add(label)
        
        # Add entity mention
        if entity_text not in entities_by_label[label]:
            entities_by_label[label].append(entity_text)
    
    return text, dict(entities_by_label), entity_types

# Test on first task
if tasks:
    text, entities, types = extract_entities_from_labelstudio(tasks[0])
    print("\n📝 Example from first document:")
    print(f"  Text length: {len(text)} characters")
    print(f"  Entity types found: {len(types)}")
    print(f"  Entity types: {sorted(types)}")
    print(f"\n  Sample entities:")
    for entity_type, mentions in list(entities.items())[:3]:
        print(f"    {entity_type}: {mentions[:3]}")


📝 Example from first document:
  Text length: 2511 characters
  Entity types found: 14
  Entity types: ['ADDRESS', 'CITY', 'DATE', 'FILING_REFERENCE', 'GOVERNING_BODY', 'GPE', 'HONORIFIC', 'MANDATE', 'MEETING_SUBJECT', 'OCCUPATION', 'OFFICE_TYPE', 'ORGANIZATION', 'ORGANIZATION_TYPE', 'PERSON']

  Sample entities:
    DATE: ['neuf février', 'vingt-six janvier dernier', 'mil neuf cent vingt-huit']
    MEETING_SUBJECT: ['NOMINATIONS']
    ORGANIZATION: ['Société anonyme belge de la Manufacture de Bethune']


In [5]:
def create_gliner2_example(text: str, entities: Dict[str, List[str]]) -> dict:
    """
    Create a GLINER2 training example in the correct format.
    
    Args:
        text: The input text
        entities: Dictionary mapping entity types to lists of entity mentions
        
    Returns:
        GLINER2 training example dictionary
    """
    return {
        "input": text,
        "output": {
            "entities": entities
        }
    }

# Test conversion
if tasks:
    text, entities, types = extract_entities_from_labelstudio(tasks[0])
    example = create_gliner2_example(text, entities)
    
    print("\n📋 GLINER2 Example Format:")
    print(json.dumps(example, indent=2, ensure_ascii=False)[:500] + "...")


📋 GLINER2 Example Format:
{
  "input": "2134. - Société anonyme belge de la Manufacture de Bethune, à Bruxelles.\n\nNOMINATIONS.\n\nL'an mil neuf cent vingt-huit, le neuf février.\nEn notre etude a Bruxelles, rue du Moniteur, numéro 8.\nDevant nous, Victor Scheyven, notaire résidant à Bruxelles.\nS'est réunie, en application de l'article quarante des statuts, l'assemblée générale extraordinaire des associés de la société anonyme Société anonyme belge de la Manufacture de Bethune, établie à Bruxellés, constituée ce jour p...


In [6]:
def convert_labelstudio_to_gliner2(tasks: List[dict]) -> List[dict]:
    """
    Convert all Label Studio tasks to GLINER2 format.
    
    Args:
        tasks: List of Label Studio tasks
        
    Returns:
        List of GLINER2 training examples
    """
    gliner2_examples = []
    skipped = 0
    
    for i, task in enumerate(tasks):
        text, entities, types = extract_entities_from_labelstudio(task)
        
        # Skip if no text or no entities
        if not text or not entities:
            skipped += 1
            continue
        
        example = create_gliner2_example(text, entities)
        gliner2_examples.append(example)
    
    print(f"\n✓ Conversion complete:")
    print(f"  Total tasks: {len(tasks)}")
    print(f"  Converted: {len(gliner2_examples)}")
    print(f"  Skipped (no entities): {skipped}")
    
    return gliner2_examples

# Convert all tasks
gliner2_data = convert_labelstudio_to_gliner2(tasks)


✓ Conversion complete:
  Total tasks: 2
  Converted: 2
  Skipped (no entities): 0


In [7]:
def save_gliner2_jsonl(examples: List[dict], filepath: str):
    """
    Save GLINER2 examples to JSONL file.
    Each line contains one JSON object (not a JSON array).
    
    Args:
        examples: List of GLINER2 training examples
        filepath: Output file path
    """
    with open(filepath, 'w', encoding='utf-8') as f:
        for example in examples:
            json_line = json.dumps(example, ensure_ascii=False)
            f.write(json_line + '\n')
    
    print(f"\n✓ Saved {len(examples)} examples to: {filepath}")
    
    # Show file size
    import os
    file_size = os.path.getsize(filepath)
    print(f"  File size: {file_size:,} bytes ({file_size/1024:.1f} KB)")

# Save to JSONL file
save_gliner2_jsonl(gliner2_data, OUTPUT_FILE)


✓ Saved 2 examples to: C:\Users\vducatte\OneDrive - UGent\Documents\BELHISFIRM\NER\gliner2-project\gliner2_training_data.jsonl
  File size: 6,962 bytes (6.8 KB)


In [8]:
def show_statistics(examples: List[dict]):
    """
    Display statistics about the converted data.
    """
    # Count entity types across all examples
    from collections import Counter
    
    entity_type_counts = Counter()
    entity_mention_counts = Counter()
    total_mentions = 0
    
    for example in examples:
        entities = example['output']['entities']
        for entity_type, mentions in entities.items():
            entity_type_counts[entity_type] += 1
            entity_mention_counts[entity_type] += len(mentions)
            total_mentions += len(mentions)
    
    print("\n" + "="*60)
    print("📊 DATASET STATISTICS")
    print("="*60)
    print(f"\nTotal examples: {len(examples)}")
    print(f"Total entity mentions: {total_mentions}")
    print(f"Unique entity types: {len(entity_type_counts)}")
    
    print(f"\n{'Entity Type':<20} {'Documents':<12} {'Mentions':<10} {'Avg/Doc'}")
    print("-" * 60)
    for entity_type in sorted(entity_type_counts.keys()):
        docs = entity_type_counts[entity_type]
        mentions = entity_mention_counts[entity_type]
        avg = mentions / docs
        print(f"{entity_type:<20} {docs:<12} {mentions:<10} {avg:.1f}")

# Show statistics
show_statistics(gliner2_data)


📊 DATASET STATISTICS

Total examples: 2
Total entity mentions: 84
Unique entity types: 15

Entity Type          Documents    Mentions   Avg/Doc
------------------------------------------------------------
ADDRESS              2            12         6.0
CITY                 2            13         6.5
DATE                 2            8          4.0
FILING_REFERENCE     2            2          1.0
GOVERNING_BODY       1            2          2.0
GPE                  1            1          1.0
HONORIFIC            2            3          1.5
MANDATE              2            3          1.5
MEETING_SUBJECT      2            2          1.0
OCCUPATION           2            10         5.0
OFFICE_TYPE          1            1          1.0
ORGANIZATION         2            4          2.0
ORGANIZATION_ACTIVITY 1            1          1.0
ORGANIZATION_TYPE    2            4          2.0
PERSON               2            18         9.0


In [17]:
def validate_gliner2_format(filepath: str) -> bool:
    """
    Validate that the JSONL file has correct GLINER2 format.
    
    Args:
        filepath: Path to JSONL file
        
    Returns:
        True if valid, False otherwise
    """
    print("\n" + "="*60)
    print("🔍 VALIDATION")
    print("="*60)
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        print(f"\n✓ File has {len(lines)} lines")
        
        # Check each line is valid JSON
        errors = []
        for i, line in enumerate(lines):
            try:
                data = json.loads(line)
                
                # Check required fields
                if 'input' not in data:
                    errors.append(f"Line {i+1}: Missing 'input' field")
                if 'output' not in data:
                    errors.append(f"Line {i+1}: Missing 'output' field")
                elif 'entities' not in data['output']:
                    errors.append(f"Line {i+1}: Missing 'entities' in output")
                
                # Check entities is a dict
                if 'output' in data and 'entities' in data['output']:
                    if not isinstance(data['output']['entities'], dict):
                        errors.append(f"Line {i+1}: 'entities' must be a dict")
                
            except json.JSONDecodeError as e:
                errors.append(f"Line {i+1}: Invalid JSON - {e}")
        
        if errors:
            print(f"\n❌ Found {len(errors)} errors:")
            for error in errors[:10]:  # Show first 10 errors
                print(f"  • {error}")
            if len(errors) > 10:
                print(f"  ... and {len(errors) - 10} more")
            return False
        else:
            print("✓ All lines are valid JSON")
            print("✓ All required fields present")
            print("✓ Format is correct")
            return True
            
    except Exception as e:
        print(f"❌ Validation failed: {e}")
        return False

# Validate the output file
is_valid = validate_gliner2_format(OUTPUT_FILE)


🔍 VALIDATION

✓ File has 2 lines
✓ All lines are valid JSON
✓ All required fields present
✓ Format is correct


In [18]:
# Detailed preview of a complete example
print("\n" + "="*60)
print("🔍 DETAILED EXAMPLE STRUCTURE")
print("="*60)

example = gliner2_data[0]

print("\n📥 INPUT (first 300 characters):")
print("-" * 60)
print(example['input'][:300] + "...")

print("\n\n📤 OUTPUT (entities):")
print("-" * 60)
entities = example['output']['entities']
print(f"Total entity types: {len(entities)}")
print()

for entity_type, mentions in sorted(entities.items()):
    print(f"  {entity_type}:")
    # Show first 5 mentions for each type
    for mention in mentions[:5]:
        print(f"    - \"{mention}\"")
    if len(mentions) > 5:
        print(f"    ... and {len(mentions) - 5} more")
    print()

print("\n✅ Format confirmed: Each line has 'input' (text) and 'output' (entities)")


🔍 DETAILED EXAMPLE STRUCTURE

📥 INPUT (first 300 characters):
------------------------------------------------------------
2134. - Société anonyme belge de la Manufacture de Bethune, à Bruxelles.

NOMINATIONS.

L'an mil neuf cent vingt-huit, le neuf février.
En notre etude a Bruxelles, rue du Moniteur, numéro 8.
Devant nous, Victor Scheyven, notaire résidant à Bruxelles.
S'est réunie, en application de l'article quarant...


📤 OUTPUT (entities):
------------------------------------------------------------
Total entity types: 14

  ADDRESS:
    - "rue du Faubourg d'Arras"
    - "place du Panthéon, numéro 1"
    - "Grand'-
Place, numéro 50"
    - "Route Nationale"
    - "rue du Moniteur, numéro 8"

  CITY:
    - "Bruxelles"
    - "Paris"
    - "Nœux-les-Mines"
    - "Arras"
    - "Bethune"
    ... and 1 more

  DATE:
    - "neuf février"
    - "vingt-six janvier dernier"
    - "mil neuf cent vingt-huit"
    - "vingt-sept janvier dernier"

  FILING_REFERENCE:
    - "2134"

  GOVERNING_BO